### Import dependencies and load data

In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

Define constants

In [6]:
TEST_SIZE = 0.2
FIRST_LAYER_NEURONS = 128
SECOND_LAYER_NEURONS = 64

In [7]:
processed_data_path = "../data/processed/"
segmented_mfccs = np.load(processed_data_path+"segmented_mfccs.npy")
onehot_labels = np.load(processed_data_path+"segmented_onehot_labels.npy")

Split data set

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    segmented_mfccs, onehot_labels, test_size=TEST_SIZE, random_state=42)

### Model architecture

Define the model

In [9]:
model = keras.Sequential([
    # Input shape based on MFCC features
    layers.Input(shape=X_train.shape[1:]),
    layers.Flatten(),  # Flatten the input
    # Dense layer with ReLU activation
    layers.Dense(FIRST_LAYER_NEURONS, activation='relu'),
    # Dense layer with ReLU activation
    layers.Dense(SECOND_LAYER_NEURONS, activation='relu'),
    # Output layer with softmax activation
    layers.Dense(y_train.shape[1], activation='softmax')
])

Compile the model

In [10]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
9031/9031 ━━━━━━━━━━━━━━━━━━━━ 35s 4ms/step - accuracy: 0.4347 - loss: 1.4573 - val_accuracy: 0.5033 - val_loss: 1.0139
Epoch 2/10
9031/9031 ━━━━━━━━━━━━━━━━━━━━ 43s 5ms/step - accuracy: 0.5415 - loss: 0.9357 - val_accuracy: 0.5835 - val_loss: 0.8727
Epoch 3/10
9031/9031 ━━━━━━━━━━━━━━━━━━━━ 44s 5ms/step - accuracy: 0.5848 - loss: 0.8714 - val_accuracy: 0.5975 - val_loss: 0.8502
Epoch 4/10
9031/9031 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.6058 - loss: 0.8370 - val_accuracy: 0.6082 - val_loss: 0.8358
Epoch 5/10
9031/9031 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - accuracy: 0.6175 - loss: 0.8202 - val_accuracy: 0.6154 - val_loss: 0.8284
Epoch 6/10
9031/9031 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - accuracy: 0.6286 - loss: 0.8070 - val_accuracy: 0.6242 - val_loss: 0.8075
Epoch 7/10
9031/9031 ━━━━━━━━━━━━━━━━━━━━ 40s 4ms/step - accuracy: 0.6337 - loss: 0.7950 - val_accuracy: 0.6273 - val_loss: 0.8059
Epoch 8/10
9031/9031 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.6392 - loss: 0

### Evaluate the model on training set

In [12]:
# Evaluate the model on training set
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print(f"Train loss: {train_loss}")
print(f"Train accuracy: {train_accuracy}")

11289/11289 ━━━━━━━━━━━━━━━━━━━━ 14s 1ms/step - accuracy: 0.6452 - loss: 0.7734
Train loss: 0.7764433026313782
Train accuracy: 0.644606351852417


### Evaluate the model on the testing set

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


2823/2823 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.6466 - loss: 0.7824
Test Loss: 0.7776288986206055
Test Accuracy: 0.6483954787254333
